# Point by Ref Analysis

In [9]:
from src.fantasy_football_v2.data.generate_dataframes.generate_weekly_points_df import generate_weekly_points_df
from src.fantasy_football_v2.data.generate_dataframes.generate_ref_df import generate_ref_df
from src.fantasy_football_v2.data.generate_dataframes.generate_game_df import generate_game_df

In [5]:
# Ref dataframe with current refs
ref_df = generate_ref_df(
    years=[2025]
).reindex()
ref_df.head()

,id,name,season,position
48910,SmitSh0r,Shawn Smith,2025,R
48911,BlakTr0r,Tra Blake,2025,U
48912,BilbJa0r,Jay Bilbo,2025,DJ
48913,SeemJe0r,Jeff Seeman,2025,LJ
48914,PagaDi0r,Dino Paganelli,2025,BJ


In [6]:
ref_df.shape

(120, 4)

In [7]:
# Dataframe of current refs
points_df = generate_weekly_points_df().reindex()
points_df.shape

Downcasting floats.


(26786, 8)

In [8]:
points_df.head()

,season,week,player_id,player_display_name,recent_team,position,fantasy_points_ppr,game_id
0,2024,1,00-0023459,Aaron Rodgers,NYJ,QB,8.580000,2024_01_NYJ_SF
1,2024,2,00-0023459,Aaron Rodgers,NYJ,QB,15.140000,2024_02_NYJ_TEN
2,2024,3,00-0023459,Aaron Rodgers,NYJ,QB,21.040001,2024_03_NE_NYJ
3,2024,4,00-0023459,Aaron Rodgers,NYJ,QB,11.600000,2024_04_DEN_NYJ
4,2024,5,00-0023459,Aaron Rodgers,NYJ,QB,11.760000,2024_05_NYJ_MIN


In [ ]:
# Merge points_df with ref_df to only include current refs
points_df = ref_df.merage(
    points_df[['referee']]
)